# Pengembangan Aplikasi Bagi Para Pecinta Buku

Virus corona yang kehadirannya mengejutkan seluruh dunia telah mengubah rutinitas harian semua orang. Kini, warga kota tidak lagi menghabiskan waktu luangnya di luar rumah seperti pergi ke kafe dan mal. Mereka lebih sering berada di rumah, menghabiskan waktunya dengan membaca buku. Hal ini pun mendorong perusahaan *startup* untuk mengembangkan aplikasi baru bagi para pecinta buku.

Kita telah memiliki sebuah database dari salah satu perusahaan yang bersaing dalam industri ini. Database tersebut berisi data tentang buku, penerbit, penulis, serta rating dan ulasan pelanggan atas buku terkait. Informasi ini akan digunakan dalam membuat penawaran harga untuk sebuah produk baru.

## Pendahuluan

Dalam proyek kali ini, kita akan mempelajari dan menganalisis database yang berisi mengenai data tentang buku, penerbit, penulis, serta rating dan ulasan pelanggan atas buku terkait supaya kita mendapatkan informasi mengenai faktor-faktor yang berpengaruh dalam industri buku sehingga dapat membuat penawaran harga untuk sebuah produk baru. 

### Tujuan
Menganalisis data untuk mengetahui faktor-faktor apa saja yang dapat berpengaruh dalam industri atau pasar buku, sehingga kita dapat memberikan usulan harga yang kompetitif untuk produk baru yang ditujukan kepada para pecinta buku.


### Tahapan
Kumpulan data yang kita miliki dari database berikut: https://storage.yandexcloud.net/cloud-certs/CA.pem, terbagi dalam 5 tabel, yaitu:

1. Data mengenai buku pada tabel `books`
2. Data mengenai pengarang/penulis pada tabel `authors`
3. Data mengenai rating pada tabel `ratings
4. Data mengenai ulasan pada tabel `reviews`
5. Data mengenai penerbit pada tabel `publishers`

Pertama, kita akan mempelajari isi datanya, dan kemudian melakukan analisis data, sebelum akhirnya menarik kesimpulan.


In [105]:
tables = ['books', 'authors', 'publishers', 'ratings', 'reviews']

for table in tables:
    query = f"SELECT column_name, data_type FROM information_schema.columns WHERE table_name = '{table}'"
    print(f"Table {table}:")
    display(run_sql_query(query, engine))

Table books:


,column_name,data_type
0,book_id,integer
1,author_id,integer
2,title,text
3,num_pages,integer
4,publication_date,date
5,publisher_id,integer


Table authors:


,column_name,data_type
0,author_id,integer
1,author,text


Table publishers:


,column_name,data_type
0,publisher_id,integer
1,publisher,text


Table ratings:


,column_name,data_type
0,rating_id,integer
1,book_id,integer
2,username,text
3,rating,integer


Table reviews:


,column_name,data_type
0,review_id,integer
1,book_id,integer
2,username,text
3,text,text


## Eksplorasi Data

### Memuat Data

In [86]:
# impor library yang diperlukan
import pandas as pd
from sqlalchemy import create_engine

db_config = {'user': 'practicum_student',         # nama pengguna
             'pwd': 's65BlTKV3faNIGhmvJVzOqhs', # kata sandi
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432,              # port koneksi
             'db': 'data-analyst-final-project-db'}          # nama database

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                                     db_config['pwd'],
                                                                       db_config['host'],
                                                                       db_config['port'],
                                                                       db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [87]:
def run_sql_query(query, engine):
    return pd.io.sql.read_sql(query, con = engine)

### Mempersiapkan Data

**DESKRIPSI DATA**

Tabel `books`:
Berisi data tentang buku:
 * book_id — ID buku
 * author_id — ID penulis
 * title — judul buku
 * num_pages — jumlah halaman
 * publication_date — tanggal penerbitan
 * publisher_id — ID penerbit
 
Tabel `authors`:
Berisi data tentang penulis:
 * author_id — ID penulis
 * author — nama penulis

Tabel `publishers`:
Berisi data tentang penerbit:
 * publisher_id — ID penerbit
 * publisher — nama penerbit

Tabel `ratings`:
Berisi data tentang ulasan pengguna:
 * rating_id — ID rating
 * book_id — ID buku
 * username — nama pengguna yang memberi rating buku
 * rating

Tabel `reviews`:
Berisi data tentang ulasan pelanggan:
 * review_id — ID ulasan
 * book_id — ID buku
 * username — nama pengguna yang mengulas buku
 * text — teks ulasan

In [88]:
tables = ['books', 'authors', 'publishers', 'ratings', 'reviews']

for table in tables:
    query = 'SELECT * FROM ' + table + ' LIMIT 10'
    display(run_sql_query(query, engine))

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268
5,6,257,1st to Die (Women's Murder Club #1),424,2005-05-20,116
6,7,258,2nd Chance (Women's Murder Club #2),400,2005-05-20,116
7,8,260,4th of July (Women's Murder Club #4),448,2006-06-01,318
8,9,563,A Beautiful Mind,461,2002-02-04,104
9,10,445,A Bend in the Road,341,2005-04-01,116


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd
5,6,Alan Paton
6,7,Albert Camus/Justin O'Brien
7,8,Aldous Huxley
8,9,Aldous Huxley/Christopher Hitchens
9,10,Aleksandr Solzhenitsyn/H.T. Willetts


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company
5,6,Aladdin
6,7,Aladdin Paperbacks
7,8,Albin Michel
8,9,Alfred A. Knopf
9,10,Alfred A. Knopf Books for Young Readers


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2
5,6,3,johnsonamanda,4
6,7,3,scotttamara,5
7,8,3,lesliegibbs,5
8,9,4,abbottjames,5
9,10,4,valenciaanne,4


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...
5,6,3,lesliegibbs,Analysis no several cause international.
6,7,4,valenciaanne,One there cost another. Say type save. With pe...
7,8,4,abbottjames,Within enough mother. There at system full rec...
8,9,5,npowers,Thank now focus realize economy focus fly. Ite...
9,10,5,staylor,Game push lot reduce where remember. Including...


### Nilai yang Hilang

In [89]:
query = '''SELECT  
book_id,
author_id,
title,
num_pages,
publication_date,
publisher_id
FROM books
WHERE
book_id IS NULL OR author_id IS NULL OR title IS NULL OR num_pages IS NULL 
OR publication_date IS NULL OR publisher_id IS NULL;'''
result = run_sql_query(query, engine)
result

,book_id,author_id,title,num_pages,publication_date,publisher_id


In [90]:
query = '''SELECT  
author_id,
author
FROM authors
WHERE
author_id IS NULL OR author IS NULL;'''
result = run_sql_query(query, engine)
result

,author_id,author


In [91]:
query = '''SELECT  
publisher_id,
publisher
FROM publishers
WHERE
publisher_id IS NULL OR publisher IS NULL;'''
result = run_sql_query(query, engine)
result

,publisher_id,publisher


In [92]:
query = '''SELECT  
    rating_id,
    book_id,
    username,
    rating
FROM ratings
WHERE
    rating_id IS NULL OR book_id IS NULL OR username IS NULL OR rating IS NULL;'''
result = run_sql_query(query, engine)
result

,rating_id,book_id,username,rating


In [93]:
query = '''SELECT  
    review_id,
    book_id,
    username,
    text
FROM reviews
WHERE
    review_id IS NULL OR book_id IS NULL OR username IS NULL OR text IS NULL;'''
result = run_sql_query(query, engine)
result

,review_id,book_id,username,text


### Jumlah Baris dan Kolom

In [94]:
#Menghitung jumlah baris dan kolom
tables = ['books', 'authors', 'publishers', 'ratings', 'reviews']

for table in tables:
    query_0 = f"SELECT '{table}' AS table_name, (SELECT COUNT(*) FROM {table}) AS row_count, COUNT(column_name) AS column_count FROM information_schema.columns WHERE table_name = '{table}'"
    display(run_sql_query(query_0, engine))


,table_name,row_count,column_count
0,books,1000,6


,table_name,row_count,column_count
0,authors,636,2


,table_name,row_count,column_count
0,publishers,340,2


,table_name,row_count,column_count
0,ratings,6456,4


,table_name,row_count,column_count
0,reviews,2793,4


### Mengatasi Duplikat

In [95]:
query="""SELECT book_id, author_id, title, num_pages, publication_date, publisher_id, COUNT(*)
                FROM books
            GROUP BY book_id, author_id, title, num_pages, publication_date, publisher_id
                HAVING COUNT(*) > 1"""
result = run_sql_query(query, engine)
result

,book_id,author_id,title,num_pages,publication_date,publisher_id,count


In [96]:
query="""SELECT author_id, author, COUNT(*)
                FROM authors
            GROUP BY author_id, author
                HAVING COUNT(*) > 1"""
result = run_sql_query(query, engine)
result

,author_id,author,count


In [97]:
query="""SELECT publisher_id, publisher, COUNT(*)
                FROM publishers
            GROUP BY publisher_id, publisher
                HAVING COUNT(*) > 1"""
result = run_sql_query(query, engine)
result

,publisher_id,publisher,count


In [98]:
query="""SELECT rating_id, book_id, username, rating, COUNT(*)
                FROM ratings
            GROUP BY rating_id, book_id, username, rating
                HAVING COUNT(*) > 1"""
result = run_sql_query(query, engine)
result

,rating_id,book_id,username,rating,count


In [99]:
query="""SELECT review_id, book_id, username, text, COUNT(*)
                FROM reviews
            GROUP BY review_id, book_id, username, text
                HAVING COUNT(*) > 1"""
result = run_sql_query(query, engine)
result

,review_id,book_id,username,text,count


### Kesimpulan dan Langkah Selanjutnya

Database yang kita miliki berisi 5 tabel dengan pembagian sebagai berikut:
 * tabel `books` yang berisi data tentang buku memiliki 1000 baris dan 6 kolom.
 * tabel `authors` yang berisi data tentang penulis memiliki 636 baris dan 2 kolom.
 * tabel `publishers` yang berisi data tentang penerbit memiliki 340 baris dan 2 kolom.
 * tabel `ratings` yang berisi data tentang rating memiliki 6456 baris dan 4 kolom.
 * tabel `reviews` yang berisi data tentang ulasan memiliki 2793 baris dan 4 kolom.
 
Tidak terdapat nilai yang hilang dan tidak ada data yang terduplikasi pada data yang kita miliki, tipe datanya pun sudah sesuai.

Selanjutnya, kita akan melakukan analisa data.

## Analisa Data

### Jumlah Buku yang Dirilis Setelah Tanggal 1 Januari 2000

In [100]:
query = ''' SELECT 
COUNT(books.book_id) as books_count 
FROM books 
WHERE 
publication_date::date > '2000-01-01'; '''
result = run_sql_query(query, engine)
result

,books_count
0,819


Terdapat 819 buku yang dirilis setelah tanggal 1 Januari 2000. Jumlah ini cukup banyak mengingat persentasenya sekitar 82% dari jumlah data buku yang kita miliki.

### Jumlah Ulasan Pengguna dan Rating Rata-rata untuk Setiap Buku

In [101]:
query = """SELECT
              books.title,
              subquery.avg_rating,
              subquery.review_count
              FROM
                  (SELECT
                       reviews.book_id as book_id,
                       COUNT (DISTINCT reviews.review_id) AS review_count,
                       AVG (ratings.rating) AS avg_rating
                   FROM
                       reviews
                       INNER JOIN ratings ON ratings.book_id = reviews.book_id
                   GROUP BY
                       reviews.book_id
                   ORDER BY
                       review_count DESC,
                       avg_rating DESC) AS subquery
                   INNER JOIN books ON subquery.book_id = books.book_id
              ORDER BY
                  review_count DESC,
                  avg_rating DESC
              LIMIT 10"""

result = run_sql_query(query, engine)
result

,title,avg_rating,review_count
0,Twilight (Twilight #1),3.662500,7
1,Harry Potter and the Prisoner of Azkaban (Harr...,4.414634,6
2,Harry Potter and the Chamber of Secrets (Harry...,4.287500,6
3,The Book Thief,4.264151,6
4,The Glass Castle,4.206897,6
5,The Hobbit or There and Back Again,4.125000,6
6,Outlander (Outlander #1),4.125000,6
7,The Curious Incident of the Dog in the Night-Time,4.081081,6
8,The Lightning Thief (Percy Jackson and the Oly...,4.080645,6
9,Water for Elephants,3.977273,6


Setelah menghitung jumlah ulasan dan rata-rata rating untuk setiap judul buku secara individu, kita dapat melihat buku-buku mana yang cukup populer (mendapat banyak ulasan dan ratingnya cukup tinggi) dan buku mana yang sepertinya tidak populer.

Buku *Twilight (Twilight #1)* memperoleh jumlah ulasan tertinggi (sebanyak 7 ulasan) namun rata-rata ratingnya hanya 3.7 saja, sedangkan buku *Harry Potter and the Prisoner of Azkaban* dan *Harry Potter and the Chamber of Secrets*, masing-masing hanya memiliki 6 ulasan, namun ratingnya cukup tinggi (4.4 dan 4.2).

Jumlah ulasan yang lebih tinggi pada buku *Twilight (Twilight #1)* (7 ulasan) menunjukkan bahwa penulisnya mungkin lebih populer dalam hal mendapatkan perhatian pembaca atau pengulas dibandingkan dengan penulis buku *Harry Potter and the Prisoner of Azkaban* (6 ulasan). Namun, tidak menjamin kualitas bukunya bagus (kualitas ratingnya bagus).

Hal yang sama dapat terjadi sebaliknya, nilai rating tinggi dengan jumlah ulasan sedikit tidak menjamin kualitas bukunya bagus, bisa jadi, nilai tinggi tersebut didapat hanya dari ulasan 1 orang saja.

### Penerbit dengan Jumlah Buku Terbanyak dan Lebih dari 50 Halaman

In [102]:
query = ''' SELECT
publishers.publisher,
COUNT(books.book_id) as total_books,
books.publisher_id
FROM books
INNER JOIN publishers ON books.publisher_id = publishers.publisher_id
WHERE books.num_pages > 50
GROUP BY
books.publisher_id,
publishers.publisher_id
ORDER BY total_books DESC;
'''
result = run_sql_query(query, engine)
result

,publisher,total_books,publisher_id
0,Penguin Books,42,212
1,Vintage,31,309
2,Grand Central Publishing,25,116
3,Penguin Classics,24,217
4,Bantam,19,35
...,...,...,...
329,HarperCollins Publishers Ltd,1,136
330,Simon & Schuster Paperbacks,1,271
331,Arrow Books,1,21
332,Headline Review,1,150


Terdapat 334 penerbit yang menerbitkan buku dengan jumlah lebih dari 50 halaman. Ternyata 98% penerbit cukup sukses dengan menerbitkan buku yang memiliki lebih dari 50 halaman, meskipun total buku yang diterbitkan masih bervariatif, ada yang sudah menerbitkan lebih dari 40 buku, dan masih ada yang baru menerbitkan 1 buku saja.

### Penulis dengan Rating Rata-rata Buku Tertinggi

In [103]:
query = """SELECT
                  authors.author,
                  AVG (subquery2.avg_rating) as final_avg            
              FROM
                  (SELECT
                      books.title,
                      books.author_id,
                      subquery1.avg_rating
                  FROM
                      (SELECT
                          book_id,
                          COUNT (rating_id) AS rating_cnt,
                          AVG (rating) AS avg_rating
                      FROM
                          ratings
                      GROUP BY
                          book_id
                      HAVING
                          COUNT (rating_id) > 50) AS subquery1
                      INNER JOIN books ON books.book_id = subquery1.book_id) AS subquery2
                  INNER JOIN authors ON authors.author_id = subquery2.author_id
              GROUP BY
                  author
              ORDER BY
                  final_avg DESC """
result = run_sql_query(query, engine)
result

,author,final_avg
0,J.K. Rowling/Mary GrandPré,4.283844
1,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,J.R.R. Tolkien,4.258446
3,Louisa May Alcott,4.192308
4,Rick Riordan,4.080645
5,William Golding,3.901408
6,J.D. Salinger,3.825581
7,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.789474
8,William Shakespeare/Paul Werstine/Barbara A. M...,3.787879
9,Dan Brown,3.754540


Dari sekitar 600 penulis yang ada, ternyata hanya 13 orang saja yang memiliki rating rata-rata buku tertinggi dan mendapatkan jumlah ulasan di atas 50. Rating tersebut adalah rating dari keseluruhan judul buku yang pernah ditulis oleh sang penulisnya, jadi bukan hanya dari 1 judul buku saja (kecuali penulis tersebut hanya memiliki 1 karya/buku saja). Beberapa nama terdengar tidak asing bagi para pecinta buku, seperti *J.K. Rowling*, *J.R.R Tolkien*, *Dan Brown* dan *William Shakespeare*.

Dari analisis kita lihat sebelumnya, buku *Twilight (Twilight #1)* memiliki jumlah ulasan terbanyak, namun, ratingnya tidak begitu bagus. Penulisnya pun (*Stephenie Meyer*) hanya memiliki nilai rata-rata rating 3.7 saja untuk keseluruhan bukunya.
Ternyata, penulis dengan rata-rata buku tertinggi adalah *J.K. Rowling*, sang penulis buku *Harry Potter Series* dengan rata-rata rating 4.3.

Rating yang tinggi bisa menjadi indikator kepuasan pembaca dan kualitas buku, sehingga jika seorang penulis mendapatkan nilai rating yang cukup tinggi untuk karyanya, dengan kata lain, bukunya mendapatkan penilaian positif dari para pengulas.

### Jumlah Rata-rata Teks Ulasan Pengguna yang Memberi Rating Terhadap Lebih dari 50 Buku

In [104]:
query = '''SELECT
reviews.username,
AVG(LENGTH(reviews.text)) as avg_text,
COUNT (ratings.rating) as total_rate
FROM reviews
INNER JOIN ratings ON ratings.book_id = reviews.book_id
GROUP BY
reviews.username
HAVING 
COUNT(ratings.rating) > 50
ORDER BY avg_text DESC;'''
result = run_sql_query(query, engine)
result

,username,avg_text,total_rate
0,abigailmoore,136.835897,195
1,edwardbrown,126.119171,193
2,carriehale,119.333333,144
3,kingandrew,118.103704,135
4,williamswesley,117.721311,122
...,...,...,...
151,zjohnston,62.110553,199
152,roberto26,61.552083,96
153,colin90,60.849398,166
154,qadams,59.747475,99


Untuk menghitung rata-rata jumlah teks ulasan dari pengguna yang telah memberi ulasan pada lebih dari 50 buku, kita melakukan penghitungan berdasarkan jumlah karakter huruf dalam teks ulasan.
Analisis ini dapat menunjukkan seberapa aktif pembaca dalam memberikan ulasan teks dan dapat memberikan informasi mengenai tentang tingkat keterlibatan pembaca dalam memberikan masukan dan ulasan yang lebih mendalam.
Ternyata, jumlah orang yang gemar membaca buku masih tergolong sedikit, bahkan tidak mencapai 10%, dari sekitar 2000 *user* yang memberikan ulasan, hanya terdapat 156 *user* saja yang sudah memberi ulasan (atau dengan kata lain sudah membaca buku) lebih dari 50.

Contohnya, *user abigailmoore* menulis ulasan sebanyak 195 dengan rata-rata jumlah teksnya 137, namun, *user zjohnston* menulis ulasan sebanyak 199 dengan rata-rata jumlah teksnya hanya 62. Disini dapat kita lihat *user abigailmoore* mungkin lebih detail dalam menuliskan ulasannya, meskipun jumlah buku yang diulasnya tidak lebih banyak daripada *user zjohnston*.

## Kesimpulan Umum

Setelah mengolah data dan menganalisanya, kita dapat mengambil beberapa kesimpulan:

Database yang kita miliki berisi 5 tabel dengan pembagian sebagai berikut:
 * tabel `books` yang berisi data tentang buku memiliki 1000 baris dan 6 kolom.
 * tabel `authors` yang berisi data tentang penulis memiliki 636 baris dan 2 kolom.
 * tabel `publishers` yang berisi data tentang penerbit memiliki 340 baris dan 2 kolom.
 * tabel `ratings` yang berisi data tentang rating memiliki 6456 baris dan 4 kolom.
 * tabel `reviews` yang berisi data tentang ulasan memiliki 2793 baris dan 4 kolom.
 
Tidak terdapat nilai yang hilang dan tidak ada data yang terduplikasi pada data yang kita miliki, tipe datanya pun sudah sesuai.

Tahap analisa data yang kita lakukan, terdiri dari beberapa langkah dengan kesimpulannya sebagai berikut:

1. Terdapat 819 buku yang dirilis setelah tanggal 1 Januari 2000. Jumlah ini cukup banyak mengingat persentasenya sekitar 82% dari jumlah data buku yang kita miliki (1000).

2. Setelah menghitung jumlah ulasan dan rata-rata rating untuk setiap judul buku secara individu, kita dapat melihat buku-buku mana yang cukup populer (mendapat banyak ulasan dan ratingnya cukup tinggi) dan buku mana yang sepertinya tidak populer. Terkadang, buku dengan jumlah ulasan terbanyak tidak menentukan kualitas ratingnya bagus. Begitu pula sebaliknya, nilai rating tinggi dengan jumlah ulasan sedikit tidak menjamin kualitas bukunya bagus, bisa jadi, nilai tinggi tersebut didapat hanya dari ulasan 1 orang saja. Jadi, kita harus bisa mempertimbangkan kedua faktor tersebut untuk menentukan buku yang cukup bagus dan populer.

3. Terdapat 334 penerbit yang menerbitkan buku dengan jumlah lebih dari 50 halaman. Ternyata 98% penerbit cukup sukses dengan menerbitkan buku yang memiliki lebih dari 50 halaman, meskipun total buku yang diterbitkan masih bervariatif, ada yang sudah menerbitkan lebih dari 40 buku, dan masih ada yang baru menerbitkan 1 buku saja.

4. Dari sekitar 600 penulis yang ada, ternyata hanya 13 orang saja yang memiliki rating rata-rata buku tertinggi dan mendapatkan jumlah ulasan di atas 50. Rating tersebut adalah rating dari keseluruhan judul buku yang pernah ditulis oleh sang penulisnya, jadi bukan hanya dari 1 judul buku saja (kecuali penulis tersebut hanya memiliki 1 karya/buku saja). Penulis dengan rata-rata buku tertinggi adalah *J.K. Rowling*, sang penulis buku *Harry Potter Series* dengan rata-rata rating 4.3. Dari analisis kita sebelumnya, ada buku yang memiliki jumlah ulasan terbanyak (sebut saja buku A), namun, ratingnya tidak begitu bagus. Jumlah ulasan yang lebih tinggi pada buku tersebut menunjukkan bahwa penulisnya mungkin lebih populer dalam hal mendapatkan perhatian pembaca atau pengulas dibandingkan dengan penulis buku dengan rata-rata rating tertinggi, namun ulasannya tidak sebanyak buku A. Namun, rating yang tinggi biasanya dapat menjadi indikator kepuasan pembaca dan kualitas buku. Maka, kita dapat mengatakan bahwa penulis dengan rata-rata penilaian tertinggi, sepertinya belum tentu memiliki popularitas yang lebih besar di kalangan pembaca, tetapi bukunya mendapatkan penilaian lebih positif dan kualitasnya baik.

5. Untuk menghitung rata-rata jumlah teks ulasan dari pengguna yang telah memberi ulasan pada lebih dari 50 buku, kita melakukan penghitungan berdasarkan jumlah karakter huruf dalam teks ulasan. Analisis ini dapat menunjukkan seberapa aktif pembaca dalam memberikan ulasan teks dan dapat memberikan informasi mengenai tentang tingkat keterlibatan pengguna dalam memberikan masukan dan ulasan yang lebih mendalam. Pembaca yang memiliki rata-rata jumlah teks lebih banyak dapat dikatakan sebagai pembaca yang lebih detail dalam menuliskan ulasan, dibandingkan pembaca yang sudah menuliskan ulasan untuk total buku lebih banyak, namun rata-rata jumlah teksnya lebih sedikit. Hal ini juga dapat menunjukkan bagaimana pembaca benar-benar membaca bukunya atau tidak, bisa saja jumlah teks yang sangat sedikit menunjukkan bahwa pembaca tersebut hanya memberi ulasan ala kadarnya karena tidak benar-benar membaca bukunya, atau mungkin karena tidak menikmati buku tersebut.

Dengan demikian, kita dapat melihat beberapa faktor yang dapat berpengaruh terhadap tren industri buku, preferensi pembaca, kualitas buku, dan keterlibatan pengguna.
Dengan menghitung jumlah ulasan dan rata-rata rating untuk setiap judul buku, kita dapat menjadikannya acuan untuk menganalisa lebih dalam mengenai buku-buku yang berpotensi menarik minat pembaca dan juga buku yang sebaiknya dihindari dalam pengembangan produk baru. Selain itu, kita juga mendapatkan informasi mengenai para penulis yang memiliki reputasi baik dan kualitas tulisannya (bukunya) yang dihargai oleh pembaca, sehingga dapat menjadi daya tarik dalam produk baru.
Kita juga mengetahui bagaimana peran para penerbit dalam industri buku, hal ini dapat membantu ketika kita menjalin kemitraan dengan para penerbit tersebut. Mungkin kita bisa bekerja dengan penerbit terkenal tetapi dengan harga mahal, atau bekerjasama dengan penerbit yang belum populer namun dengan harga murah. Masing-masing memiliki keuntungan dan kerugian tersendiri dalam bisnis kita untuk produk baru.
Dengan memperhatikan ulasan yang ditulis oleh pembaca, kita dapat menentukan *target* pemasaran produk baru kita. Dengan menargetkan para pembaca yang aktif dan terlibat dalam memberikan ulasan teks yang lebih detail, pemasaran produk baru dapat difokuskan pada mereka yang cenderung memberikan kontribusi yang bernilai dan menghasilkan ulasan yang lebih mendalam, selain itu, dapat membantu meningkatkan kualitas dan informasi yang diberikan oleh pengguna lain dalam komunitas aplikasi buku, serta membangun kesetiaan dan partisipasi aktif dalam aplikasi tersebut.